# Goal: Upload a table with the following columns to our Heroku database.
Our raw data is already uploaded to Heroku (code in "upload_data.ipynb"). Now, we will transform it in accordance with the following definitions.

### Column Definitions

- order_id: the id of the order
- company_id: the id of the company for which the order was performed
- origin_city: the city where the order was picked up
- destination_city: the city where the order was delivered
- pick_up_time_local: the time of pickup in the time zone of the pickup location
- delivery_time_local: the time of delivery in the time zone of the delivery location
- minutes_to_pickup: how many minutes between order creation and pickup
- order_type: drive, hfpu, or nfo. A drive has a single driving segment, hold for pickup(hfpu) will have just a drive and a flight, and nfo will have a drive, flight, and another drive
- total_drive_distance: sum of the distance in miles of all of the drives associated with an order


### Other Definitions
- NFO: fastest type of shipping. This is where a driver will pick something up, put it on a plane and the a driver will recover from the destination airport and perform the final delivery.
- order: top level object
- delivery_route_segment: the modes of transit for an order. One order will have between one and three segments depending on the type of order. For instance, a drive will only have a DrivingSegment while and NFO will have A DrivingSegment, FlyingSegment, and another DrivingSegment
- driving_search: Third party response with routing information for a DrivingSegment


*Please contact Ambika Gupta at ambika.gupta97@gmail.com for database credentials.*

In [1]:
import pandas as pd
import json
import datetime
import numpy as np
%run ./read_from_heroku.ipynb ## import statement for .ipynb files

# Begin exploration of data.

### Orders

Orders holds a lot of relevant data for our goal. We will be using the "id" as order_id, created_at, pick_up_time, quoted_delivery_time, and company_id to create our final table. We'll also do some formatting on created_at, pick_up_time, and quoted_delivery_time to make using them later on easier. 

In [2]:
orders_query = """
SELECT 
    id as order_id, 
    created_at, 
    pick_up_time, 
    quoted_delivery_time, 
    company_id
FROM
    orders
"""

orders = read_table(orders_query)

orders["created_at"] = orders \
                        .apply(lambda x : pd.to_datetime(x["created_at"], format = "%Y-%m-%d %H:%M:%S") \
                        .replace(microsecond=0) if x["created_at"] else np.nan, axis = 1)
orders["pick_up_time"] = orders \
                        .apply(lambda x : pd.to_datetime(x["pick_up_time"], format = "%Y-%m-%d %H:%M:%S")
                        .replace(microsecond=0) if x["pick_up_time"] else np.nan, axis = 1)
orders["quoted_delivery_time"] = orders \
                        .apply(lambda x : pd.to_datetime(x["quoted_delivery_time"], format = "%Y-%m-%d %H:%M:%S")
                        .replace(microsecond=0) if x["quoted_delivery_time"] else np.nan, axis = 1)

orders.head()

,order_id,created_at,pick_up_time,quoted_delivery_time,company_id
0,9793,2019-09-25 18:20:17,2019-09-25 18:20:16,2019-09-26 03:02:08,210
1,9792,2019-09-25 18:13:22,2019-09-25 18:13:22,2019-09-26 03:32:20,210
2,9791,2019-09-25 18:06:23,2019-09-25 18:06:22,2019-09-26 04:30:55,365
3,9790,2019-09-25 16:23:21,2019-09-25 16:23:21,2019-09-26 04:01:26,377
4,9789,2019-09-25 16:23:00,2019-09-25 16:23:00,2019-09-26 04:01:26,377


### Start Addresses
We will use the start_addresses table to find both the origin city and timezone for each order_id.

In [3]:
start_addresses_query = """
SELECT
    id as start_address_id,
    city as origin_city,
    time_zone as origin_time_zone
FROM start_addresses
"""

start_addresses = read_table(start_addresses_query)

start_addresses.head()

,start_address_id,origin_city,origin_time_zone
0,187944,Baltimore,America/New_York
1,187732,San Diego,America/Los_Angeles
2,187754,Jacksonville Beach,America/New_York
3,182484,Burbank,America/Los_Angeles
4,182042,DFW Airport,America/Chicago


### End Addresses
We will use the end_addresses table to find both the destination city and timezone for each order_id. 

In [4]:
end_addresses_query = """
SELECT
    id as end_address_id,
    city as destination_city,
    time_zone as destination_time_zone
FROM end_addresses
"""

end_addresses = read_table(end_addresses_query)

end_addresses.head()

,end_address_id,destination_city,destination_time_zone
0,180668,Santa Barbara,America/Chicago
1,184623,CARLSBAD,America/Los_Angeles
2,185131,Carlsbad,America/Los_Angeles
3,187248,Queens,America/New_York
4,182359,Carlsbad,America/Los_Angeles


### Driving Searches
By using the driving_searches table, we can find the total_drive_distance for each order_id. The drive distance for a segment is located within the json_obj column. Again, note that there are multiple entries per order_id, so in this case, we will need to find the sum of all driving distances for each one.

In [5]:
driving_searches_query = """
SELECT
    id as driving_search_id,
    json_obj,
    order_id
FROM
    driving_searches
"""

driving_searches = read_table(driving_searches_query)

driving_searches.head()

,driving_search_id,json_obj,order_id
0,27944,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9682
1,27415,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9395
2,27559,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9476
3,27223,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9295
4,27520,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9451


### Delivery Route Segments
We will use the delivery_route_segments table to determine the order_type for each order_id based on the number of times the order_id appears in the table. We will also use the start_address_id and end_address_id to determine the origin_city and destination_city and calculate pick_up_time_local and delivery_time_local. Note that there are multiple entries per order_id, meaning we will need to determine the true origin and destination for every order. More to come later.

In [6]:
delivery_route_segments_query = """
SELECT 
    id as delivery_route_segment_id, 
    order_id, 
    start_address_id, 
    end_address_id 
FROM
    delivery_route_segments
"""

delivery_route_segments = read_table(delivery_route_segments_query)

delivery_route_segments.head()

,delivery_route_segment_id,order_id,start_address_id,end_address_id
0,22819,8761,180387,180388
1,22825,8763,180403,180404
2,22809,8758,180361,180362
3,22810,8758,180363,180364
4,22817,8761,180383,180384


# Create our output table and obtain relevant columns.

We will store our final data table in accordance with the set parameters in output. We will merge the other information we find with this table to get our final solution. Initially, we will set it equal to orders.

In [7]:
output = orders
output.head()

,order_id,created_at,pick_up_time,quoted_delivery_time,company_id
0,9793,2019-09-25 18:20:17,2019-09-25 18:20:16,2019-09-26 03:02:08,210
1,9792,2019-09-25 18:13:22,2019-09-25 18:13:22,2019-09-26 03:32:20,210
2,9791,2019-09-25 18:06:23,2019-09-25 18:06:22,2019-09-26 04:30:55,365
3,9790,2019-09-25 16:23:21,2019-09-25 16:23:21,2019-09-26 04:01:26,377
4,9789,2019-09-25 16:23:00,2019-09-25 16:23:00,2019-09-26 04:01:26,377


### Find the time until pickup for each order.
We can see that we have order_id and company_id correctly in the output table already. Let's get minutes_to_pickup simply by subtracting pick_up_time - created_at. We may encounter negative values in this calculation which means the order was created after it was picked up. We ignore these negative values as this is obviously illogical and must have been some irrelevant error.

In [8]:
output["minutes_to_pickup"] = output \
                                .apply(lambda x: max(0.0, round((x["pick_up_time"] - x["created_at"])
                                .total_seconds() / 60)), axis = 1)
output.head()

,order_id,created_at,pick_up_time,quoted_delivery_time,company_id,minutes_to_pickup
0,9793,2019-09-25 18:20:17,2019-09-25 18:20:16,2019-09-26 03:02:08,210,0.0
1,9792,2019-09-25 18:13:22,2019-09-25 18:13:22,2019-09-26 03:32:20,210,0.0
2,9791,2019-09-25 18:06:23,2019-09-25 18:06:22,2019-09-26 04:30:55,365,0.0
3,9790,2019-09-25 16:23:21,2019-09-25 16:23:21,2019-09-26 04:01:26,377,0.0
4,9789,2019-09-25 16:23:00,2019-09-25 16:23:00,2019-09-26 04:01:26,377,0.0


### Let's get the total driving distance for each order.

To do this, we create a function which will take in our JSON object from the driving_searches table and parse the dictionary for the correct distance. We call this function on every row in the driving_searches table, and we get the driving distance for each delivery route segment that was driven.

The best part is that we actually already did this when preprocessing the data before uploading it to the database. If we look in create_tables.ipynb, we can see how we utilized the function earlier and stored the data in the "distance" column.

In [9]:
## Function to get mileage from json object
def getDistance(js):
    miles = 0
    for route in js['routes']:
        for leg in route['legs']:
            dist, metric = leg['distance']['text'].split(" ")
            dist = dist.replace(",","")
            if metric == "mi":
                miles += float(dist)
            elif metric == "km":
                miles += float(dist) * 0.621
            elif metric == "ft":
                miles += float(dist) / 5280
            elif metric == "m":
                miles += float(dist) / 1609.34
                
    return(miles)
driving_searches["distance"] = driving_searches.apply(lambda x: getDistance(eval(x["json_obj"])), axis = 1)
driving_searches.head()

,driving_search_id,json_obj,order_id,distance
0,27944,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9682,80.7
1,27415,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9395,32.3
2,27559,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9476,32.8
3,27223,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9295,6.1
4,27520,"{""geocoded_waypoints"":[{""geocoder_status"":""OK""...",9451,19.8


Now, we can group the driving_search table by order_id and sum the mileage to find the total driving distance for each order. Merge this data with our output table.

In [10]:
total_miles = driving_searches.groupby('order_id').agg(total_drive_distance = ('distance', 'sum'))
total_miles.head()

,total_drive_distance
order_id,
8751,29.9
8752,48.4
8753,193.0
8754,193.0
8755,62.6


We can merge this with our output table. Now, we have found the total_drive_distance for each order!

In [11]:
output = pd.merge(output, total_miles, on = "order_id")
output.head()

,order_id,created_at,pick_up_time,quoted_delivery_time,company_id,minutes_to_pickup,total_drive_distance
0,9793,2019-09-25 18:20:17,2019-09-25 18:20:16,2019-09-26 03:02:08,210,0.0,31.1
1,9792,2019-09-25 18:13:22,2019-09-25 18:13:22,2019-09-26 03:32:20,210,0.0,32.6
2,9791,2019-09-25 18:06:23,2019-09-25 18:06:22,2019-09-26 04:30:55,365,0.0,65.8
3,9790,2019-09-25 16:23:21,2019-09-25 16:23:21,2019-09-26 04:01:26,377,0.0,58.1
4,9789,2019-09-25 16:23:00,2019-09-25 16:23:00,2019-09-26 04:01:26,377,0.0,58.1


Now, to find the rest of the columns, we must use the delivery_route_segments, start_addresses, and end_addresses tables.<br>
<br>

### Determining the true origin and destination

As noted before, in the delivery_route_segments, there are duplicates of order_id, meaning that some orders have multiple origin and destination cities. We need to determine which one is the correct value for each. We assume that the start_addresses and end_addresses are created in chronological order, meaning that the lowest value for start_address_id will hold the first location of the order and the highest for end_address_id will hold the last.<br>
<br>
In the following, we look at an example of an order_id = 9793 that has multiple delivery_route_segments associated with it.

In [12]:
example = delivery_route_segments.loc[delivery_route_segments["order_id"] == 9793]
example.head()

,delivery_route_segment_id,order_id,start_address_id,end_address_id
282,25400,9793,188051,188052
283,25398,9793,188047,188048
2444,25399,9793,188049,188050


Now, we merge this example dataframe with our start_addresses and end_addresses and we can see the corresponding cities. With a little deduction and a little help from Google Maps, we can tell that the order originated in LA, was driven to Burbank Airport, flown to SeaTac Airport, and finally was driven into Seattle. This follows our assumption, just like the rest of the data, so we will expand to the rest of the table.

In [13]:
example = pd.merge(example, start_addresses, on = "start_address_id")
example = pd.merge(example, end_addresses, on = "end_address_id")
del example["origin_time_zone"]
del example["destination_time_zone"]

example.head()

,delivery_route_segment_id,order_id,start_address_id,end_address_id,origin_city,destination_city
0,25400,9793,188051,188052,Seatac,SEATTLE
1,25398,9793,188047,188048,LOS ANGELES,Burbank
2,25399,9793,188049,188050,Burbank,Seatac


#### Expanding our assumption to the entire table.

We can execute our assumption by grouping the delivery_route_segments table on order_id. Then, we can take the minimum value of the start_address_id to find the true origin and take the maximum value of the end_address_id to find the true destination. Also, based off of the column definition of order_type, we know that 1 delivery_route segment means we have a "drive" type, 2 delivery_route_segments means a "hfpu" type, and 3 means "nfo". Therefore, we find the count of each order_id in delivery_route_segments to determine the order_type.

In [14]:
true_start_end_ids = delivery_route_segments \
    .groupby("order_id") \
    .agg({"start_address_id":"min", "end_address_id":"max", "delivery_route_segment_id":"count"}) \
    .rename(columns={"delivery_route_segment_id":"order_type"})

true_start_end_addr = pd.merge(true_start_end_ids, start_addresses, on = "start_address_id", right_index = True)
true_start_end_addr = pd.merge(true_start_end_addr, end_addresses, on = "end_address_id", right_index = True)

types = {1: "drive", 2: "hpfu", 3: "nfo"}
true_start_end_addr["order_type"] = true_start_end_addr["order_type"].map(types)

del true_start_end_addr["start_address_id"]
del true_start_end_addr["end_address_id"]

true_start_end_addr.head()

,order_type,origin_city,origin_time_zone,destination_city,destination_time_zone
order_id,,,,,
8751,nfo,FORT WORTH,America/Chicago,Tampa,America/New_York
8752,drive,Tampa,America/New_York,Winterhaven,America/New_York
8753,drive,Jamaica,America/New_York,BALLSTON SPA,America/New_York
8754,drive,Jamaica,America/New_York,BALLSTON SPA,America/New_York
8755,nfo,SAN DIEGO,America/Los_Angeles,SAXONBURG,America/New_York


Merge our table with the true origin and destination of each order with our output table. We're almost there!

In [15]:
output = pd.merge(output, true_start_end_addr, on = "order_id")
output.head()

,order_id,created_at,pick_up_time,quoted_delivery_time,company_id,minutes_to_pickup,total_drive_distance,order_type,origin_city,origin_time_zone,destination_city,destination_time_zone
0,9793,2019-09-25 18:20:17,2019-09-25 18:20:16,2019-09-26 03:02:08,210,0.0,31.1,nfo,LOS ANGELES,America/Los_Angeles,SEATTLE,America/Los_Angeles
1,9792,2019-09-25 18:13:22,2019-09-25 18:13:22,2019-09-26 03:32:20,210,0.0,32.6,nfo,LOS ANGELES,America/Los_Angeles,SEATTLE,America/Los_Angeles
2,9791,2019-09-25 18:06:23,2019-09-25 18:06:22,2019-09-26 04:30:55,365,0.0,65.8,nfo,Fort Worth,America/Chicago,Carlsbad,America/Los_Angeles
3,9790,2019-09-25 16:23:21,2019-09-25 16:23:21,2019-09-26 04:01:26,377,0.0,58.1,nfo,Carlsbad,America/Los_Angeles,Washington,America/New_York
4,9789,2019-09-25 16:23:00,2019-09-25 16:23:00,2019-09-26 04:01:26,377,0.0,58.1,nfo,Carlsbad,America/Los_Angeles,Washington,America/New_York


### Calculate local pickup and delivery times for each order.
Now, we need to use pick_up_time and quoted_delivery_time with the origin_time_zone and destination_time_zone that we just found to determine pick_up_time_local and delivery_time_local.

In [16]:
output["pick_up_time_local"] = output \
                                .apply(lambda x: x["pick_up_time"] \
                                .tz_localize("utc") \
                                .tz_convert(x["origin_time_zone"]) \
                                .replace(tzinfo = None), axis =1)

output.loc[(output["destination_time_zone"] == "Pacific Time (US & Canada)"), "destination_time_zone"] = "US/Pacific"

output["delivery_time_local"] = output \
                                .apply(lambda x: x["quoted_delivery_time"] \
                                .tz_localize("utc") \
                                .tz_convert(x["destination_time_zone"]) \
                                .replace(tzinfo = None), axis =1)

output.head()

,order_id,created_at,pick_up_time,quoted_delivery_time,company_id,minutes_to_pickup,total_drive_distance,order_type,origin_city,origin_time_zone,destination_city,destination_time_zone,pick_up_time_local,delivery_time_local
0,9793,2019-09-25 18:20:17,2019-09-25 18:20:16,2019-09-26 03:02:08,210,0.0,31.1,nfo,LOS ANGELES,America/Los_Angeles,SEATTLE,America/Los_Angeles,2019-09-25 11:20:16,2019-09-25 20:02:08
1,9792,2019-09-25 18:13:22,2019-09-25 18:13:22,2019-09-26 03:32:20,210,0.0,32.6,nfo,LOS ANGELES,America/Los_Angeles,SEATTLE,America/Los_Angeles,2019-09-25 11:13:22,2019-09-25 20:32:20
2,9791,2019-09-25 18:06:23,2019-09-25 18:06:22,2019-09-26 04:30:55,365,0.0,65.8,nfo,Fort Worth,America/Chicago,Carlsbad,America/Los_Angeles,2019-09-25 13:06:22,2019-09-25 21:30:55
3,9790,2019-09-25 16:23:21,2019-09-25 16:23:21,2019-09-26 04:01:26,377,0.0,58.1,nfo,Carlsbad,America/Los_Angeles,Washington,America/New_York,2019-09-25 09:23:21,2019-09-26 00:01:26
4,9789,2019-09-25 16:23:00,2019-09-25 16:23:00,2019-09-26 04:01:26,377,0.0,58.1,nfo,Carlsbad,America/Los_Angeles,Washington,America/New_York,2019-09-25 09:23:00,2019-09-26 00:01:26


# Time to clean up and upload our output!

### Remove all unnecessary columns and reorder.

In [17]:
output = output[["order_id", "company_id", "origin_city", "destination_city", "pick_up_time_local", 
                 "delivery_time_local", "minutes_to_pickup", "order_type", "total_drive_distance"]]

output.head()

,order_id,company_id,origin_city,destination_city,pick_up_time_local,delivery_time_local,minutes_to_pickup,order_type,total_drive_distance
0,9793,210,LOS ANGELES,SEATTLE,2019-09-25 11:20:16,2019-09-25 20:02:08,0.0,nfo,31.1
1,9792,210,LOS ANGELES,SEATTLE,2019-09-25 11:13:22,2019-09-25 20:32:20,0.0,nfo,32.6
2,9791,365,Fort Worth,Carlsbad,2019-09-25 13:06:22,2019-09-25 21:30:55,0.0,nfo,65.8
3,9790,377,Carlsbad,Washington,2019-09-25 09:23:21,2019-09-26 00:01:26,0.0,nfo,58.1
4,9789,377,Carlsbad,Washington,2019-09-25 09:23:00,2019-09-26 00:01:26,0.0,nfo,58.1


### Push table to our Heroku database.
Please contact Ambika Gupta at ambika.gupta97@gmail.com for database credentials.

In [18]:
from sqlalchemy import create_engine

database_creds = ''

if not database_creds:
    print("Please contact Ambika Gupta at ambika.gupta97@gmail.com for database credentials.")
    exit()
else:
    engine = create_engine(database_creds)
    output.to_sql('output', engine, if_exists = 'replace')
    print("Uploaded successfully to output table in Heroku!")

Uploaded successfully to output table in Heroku!
